<img src="http://csdms.colorado.edu/mediawiki/images/CSDMS_high_res_weblogo.jpg">

# Centered Parameter Study with HydroTrend

[HydroTrend](https://csdms.colorado.edu/wiki/Model:HydroTrend) is a numerical model that creates synthetic river discharge and sediment load time series as a function of climate trends and basin morphology.

In this example, we'll perform a [centered parameter study](https://dakota.sandia.gov//sites/default/files/docs/6.4/html-ref/method-centered_parameter_study.html),
evaluating how changing two HydroTrend input parameters:

* `starting_mean_annual_temperature` and
* `total_annual_precipitation`

affects two output parameters

* median long-term suspended sediment load at the river mouth and
* mean discharge at the river mouth

over a one-year run.

Before we start, make sure that you've installed Dakota, HydroTrend, and this package on your computer, using the instructions in the [README](https://github.com/csdms/dakota/blob/master/README.md) file.

Start by importing the Dakota class.

In [ ]:
from csdms.dakota import Dakota

Create a Dakota instance to perform a centered parameter study with HydroTrend.

In [ ]:
d = Dakota(method='centered_parameter_study', component='hydrotrend')

Define the HydroTrend input variables to be used in the parameter study, as well as the initial point in parameter space, the step size, and the range of the variables.

In [ ]:
d.method.variables = ['starting_mean_annual_temperature', 'total_annual_precipitation']
d.method.initial_point = [15.0, 2.0]
d.method.steps_per_variable = [2, 5]
d.method.step_vector = [2.5, 0.2]

Define the HydroTrend outputs to be used in the parameter study, as well as the statistics to be calculated from them.

In [ ]:
d.method.responses = ['Qs_median', 'Q_mean']
d.method.response_files = ['HYDROASCII.QS', 'HYDROASCII.Q']
d.method.response_statistics = ['median', 'mean']

HydroTrend requires a set of files to run.
They're included in the **data** directory of the repository containing this example.
They can also be obtained directly from the HydroTrend [GitHub repository](https://github.com/mcflugen/hydrotrend/tree/add-bmi-metadata).
Set paths to these files with the following statements.

In [ ]:
import os
data_dir = os.path.join(os.getcwd(), 'data')

template_file = os.path.join(data_dir, 'hydrotrend.in.tmpl')
parameters_file = os.path.join(data_dir, 'parameters.yaml')
hypsometry_file = os.path.join(data_dir, 'HYDRO0.HYPS')

The *template file* provides the configuration file for HydroTrend, but with all parameter values replaced by variables in the form `{parameter_name}`. The *parameters file* provides descriptions, ranges, and default values for all of the parameters represented in the template file. The *hypsometry file* describes the change in elevation along the river's course from source to sea.

From the template and parameters files,
we can create an input file that HydroTrend can run.
Included in the CSDMS Dakota package is a routine that replaces the variables in the template file with default values from the parameters file.
Import this routine and use it to create a HydroTrend input file.

In [ ]:
from csdms.dakota.plugins.base import write_dflt_file

default_input_file = write_dflt_file(template_file, parameters_file)
print default_input_file

Next, we must replace the default values for the variables for `starting_mean_annual_temperature` and `total_annual_precipitation` with variable names for Dakota to substitute into. The CSDMS Dakota package also includes a routine to do this. Import this routine and use it to create a Dakota template file.

In [ ]:
from csdms.dakota.plugins.base import write_dtmpl_file

dakota_template_file = write_dtmpl_file(template_file, default_input_file, d.method.variables)
print dakota_template_file

Associate the Dakota template file and the hypsometry file with the Dakota instance.

In [ ]:
d.method.template_file = dakota_template_file
d.method.input_files = hypsometry_file

Call the setup method to create files needed by Dakota, then run the experiment.

In [ ]:
d.setup()
d.run()

Check the output; in particular, the **dakota.dat** file.

In [ ]:
%cat dakota.dat